**Data Generation** \\
(as seen in GCNG paper)

In [ ]:
import torch

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.2.1+cu121.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.2.1+cu121.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from collections import defaultdict
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
####################  get the whole training dataset

#current_path = os.path.abspath('.')
cortex_svz_cellcentroids = pd.read_csv("cortex_svz_cellcentroids.csv")
############# get batch adjacent matrix
cell_view_list = []
for view_num in range(7):
    cell_view = cortex_svz_cellcentroids[cortex_svz_cellcentroids['Field of View']==view_num]
    cell_view_list.append(cell_view)

In [ ]:
############ the distribution of distance
distance_list_list = []
distance_list_list_2 = []
print ('calculating distance matrix, it takes a while')
for view_num in range(7):
    print (view_num)
    cell_view = cell_view_list[view_num]
    distance_list = []
    for j in range(cell_view.shape[0]):
        for i in range (cell_view.shape[0]):
            if i!=j:
                distance_list.append(np.linalg.norm(cell_view.iloc[j][['X','Y']]-cell_view.iloc[i][['X','Y']]))
    distance_list_list = distance_list_list + distance_list
    distance_list_list_2.append(distance_list)


# np.save(current_path+'/seqfish_plus/distance_array.npy',np.array(distance_list_list))

calculating distance matrix, it takes a while
0
1
2
3
4
5
6


In [ ]:
!pip install spektral

In [ ]:
###try different distance threshold, so that on average, each cell has x neighbor cells, see Tab. S1 for results
from scipy import sparse
import spektral
import pickle
import scipy.linalg
distance_array = np.array(distance_list_list)
for threshold in [140]:#[100,140,180,210,220,260]:#range (210,211):#(100,400,40):
    num_big = np.where(distance_array<threshold)[0].shape[0]
    print (threshold,num_big,str(num_big/(913*2)))
    distance_matrix_threshold_I_list = []
    distance_matrix_threshold_W_list = []
    from sklearn.metrics.pairwise import euclidean_distances
    for view_num in range (7):
        cell_view = cell_view_list[view_num]
        distance_matrix = euclidean_distances(cell_view[['X','Y']], cell_view[['X','Y']])
        distance_matrix_threshold_I = np.zeros(distance_matrix.shape)
        distance_matrix_threshold_W = np.zeros(distance_matrix.shape)
        for i in range(distance_matrix_threshold_I.shape[0]):
            for j in range(distance_matrix_threshold_I.shape[1]):
                if distance_matrix[i,j] <= threshold and distance_matrix[i,j] > 0:
                    distance_matrix_threshold_I[i,j] = 1
                    distance_matrix_threshold_W[i,j] = distance_matrix[i,j]
        distance_matrix_threshold_I_list.append(distance_matrix_threshold_I)
        distance_matrix_threshold_W_list.append(distance_matrix_threshold_W)
    whole_distance_matrix_threshold_I = scipy.linalg.block_diag(distance_matrix_threshold_I_list[0],
                                                                distance_matrix_threshold_I_list[1],
                                                                distance_matrix_threshold_I_list[2],
                                                                distance_matrix_threshold_I_list[3],
                                                                distance_matrix_threshold_I_list[4],
                                                                distance_matrix_threshold_I_list[5],
                                                                distance_matrix_threshold_I_list[6])

    '''
    ############### get normalized sparse adjacent matrix
    distance_matrix_threshold_I_N = spektral.utils.normalized_adjacency(whole_distance_matrix_threshold_I, symmetric=True)
    print(type(distance_matrix_threshold_I_N))
    distance_matrix_threshold_I_N_crs = sparse.csr_matrix(distance_matrix_threshold_I_N)
    with open('whole_FOV_distance_I_N_norm_crs_'+str(threshold), 'wb') as fp:
        pickle.dump(distance_matrix_threshold_I_N_crs, fp)'''
    ############### get not normalized sparse adjacent matrix
    distance_matrix_threshold_I_N = np.float32(whole_distance_matrix_threshold_I) ## do not normalize adjcent matrix
    print(type(distance_matrix_threshold_I_N))
    distance_matrix_threshold_I_N_crs = sparse.csr_matrix(distance_matrix_threshold_I_N)
    with open('whole_FOV_distance_I_N_crs_'+str(threshold), 'wb') as fp:
        pickle.dump(distance_matrix_threshold_I_N_crs, fp)

140 4490 2.4589266155531218
<class 'numpy.ndarray'>


In [ ]:
########### read ligand receptor database
ligand_list = pd.read_csv('ligand_list2.txt',header  = None)
receptor_list = pd.read_csv('receptor_list2.txt',header  = None)
LR_pairs = pd.read_csv('ligand_receptor_pairs2.txt',header  = None,sep ='\t')

In [ ]:
cortex_svz_counts = pd.read_csv("cortex_svz_counts.csv")
cortex_svz_counts_N =cortex_svz_counts.div(cortex_svz_counts.sum(axis=1)+1, axis='rows')*10**4
cortex_svz_counts_N.columns =[i.lower() for i in list(cortex_svz_counts_N)] ## gene expression normalization
cortex_svz_cellcentroids = pd.read_csv('cortex_svz_cellcentroids.csv')
# cortex_svz_counts_N_FOV = cortex_svz_counts_N

In [ ]:
gene_list =[i.lower() for i in list(cortex_svz_counts)]

non_LR_list = [i for i in gene_list if i not in list(ligand_list.iloc[:,0]) and i not in list(receptor_list.iloc[:,0])]
ovlp_ligand_list = [i for i in gene_list if i in list(ligand_list.iloc[:,0])]
ovlp_receptor_list = [i for i in gene_list if i in list(receptor_list.iloc[:,0])]

count = 0
h_LR = defaultdict(list)
for LR_pair_index in range(LR_pairs.shape[0]):
    ligand, receptor =  LR_pairs.iloc[LR_pair_index]
    if ligand in gene_list and receptor in gene_list:
        h_LR[ligand].append(receptor)
        count = count + 1

In [ ]:
################### generate training dataset containing both postive and negative samples, where negative samples still in the ligand and receptor set, but not in pair set
############################################# to split the LR database completely

def generate_LR_pairs (h_LR_original,sub_ligand_list, sub_receptor_list,cortex_svz_counts_N):
    h_LR = defaultdict(list)
    for ligand in h_LR_original.keys():
        if ligand in sub_ligand_list:
            for receptor in h_LR_original[ligand]:
                if receptor in sub_receptor_list:
                    h_LR[ligand].append(receptor)
    import random
    random.seed(0)
    count = 0
    gene_pair_list  = []
    X_data = []
    Y_data = []
    sub_ligand_list_ovlp = list(h_LR.keys())
    for ligand in sub_ligand_list_ovlp:
        for receptor in h_LR[ligand]:
            gene_pair_list.append(ligand + '\t' + receptor)
            cell_LR_expression = np.array(cortex_svz_counts_N[[ligand, receptor]]) # postive sample
            X_data.append(cell_LR_expression)
            Y_data.append(1)
            ############## get negative samples
            non_pair_receptor_list = [i for i in sub_receptor_list if i not in h_LR[ligand]]
            random.seed(count)
            random_receptor = random.sample(non_pair_receptor_list, 1)[0]
            gene_pair_list.append(ligand + '\t' + random_receptor)
            cell_LR_expression = np.array(cortex_svz_counts_N[[ligand, random_receptor]])
            X_data.append(cell_LR_expression)
            Y_data.append(0)
            count = count + 1
    ligand_record = sub_ligand_list_ovlp[0]
    gene_pair_index = [0]
    count = 0
    for gene_pair in gene_pair_list:
        ligand = gene_pair.split('\t')[0]
        if ligand == ligand_record:
            count = count + 1
        else:
            gene_pair_index.append(count)
            ligand_record = ligand
            count = count + 1
    gene_pair_index.append(count)
    X_data_array = np.array(X_data)
    Y_data_array = np.array(Y_data)
    gene_pair_list_array = np.array(gene_pair_list)
    gene_pair_index_array = np.array(gene_pair_index)
    return (X_data_array,Y_data_array,gene_pair_list_array,gene_pair_index_array) ## x data, y data, gene pair name, index to separate pairs by ligand genes

In [ ]:
########## ten fold cross validation data generation
ovlp_ligand_list_cons = ovlp_ligand_list
ovlp_receptor_list_cons = ovlp_receptor_list
import random
random.seed(1)
ovlp_ligand_list = random.sample(ovlp_ligand_list_cons,len(ovlp_ligand_list))
random.seed(1)
ovlp_receptor_list = random.sample(ovlp_receptor_list_cons,len(ovlp_receptor_list))
for test_indel in range(1,11): ################## ten fold cross validation
    print (test_indel)
    ######### completely separate ligand and recpetor genes as mutually  exclusive train and test set
    whole_ligand_index = [i for i in range(len(ovlp_ligand_list))]
    test_ligand = [i for i in range (int(np.ceil((test_indel-1)*0.1*len(ovlp_ligand_list))),int(np.ceil(test_indel*0.1*len(ovlp_ligand_list))))]
    train_ligand= [i for i in whole_ligand_index if i not in test_ligand]
    whole_receptor_index = [i for i in range(len(ovlp_receptor_list))]
    test_receptor = [i for i in range(int(np.ceil((test_indel - 1) * 0.1 * len(ovlp_receptor_list))),int(np.ceil(test_indel * 0.1 * len(ovlp_receptor_list))))]
    train_receptor = [i for i in whole_receptor_index if i not in test_receptor]
    X_data_array_train, Y_data_array_train, gene_pair_list_array_train, gene_pair_index_array_train = generate_LR_pairs (h_LR,np.array(ovlp_ligand_list)[train_ligand], np.array(ovlp_receptor_list)[train_receptor],cortex_svz_counts_N)
    X_data_array_test, Y_data_array_test, gene_pair_list_array_test, gene_pair_index_array_test = generate_LR_pairs(h_LR, np.array(ovlp_ligand_list)[test_ligand], np.array(ovlp_receptor_list)[test_receptor], cortex_svz_counts_N)

    if not os.path.isdir('rand_1_10fold/'):
        os.makedirs( 'rand_1_10fold/')
    np.save('rand_1_10fold/'+str(test_indel)+'_train_X_data_array.npy', X_data_array_train)
    np.save('rand_1_10fold/'+str(test_indel)+'_train_Y_data_array.npy', Y_data_array_train)
    np.save('rand_1_10fold/'+str(test_indel)+'_train_gene_pair_list_array.npy', gene_pair_list_array_train)
    np.save('rand_1_10fold/'+str(test_indel)+'_train_gene_pair_index_array.npy', gene_pair_index_array_train)
    np.save('rand_1_10fold/' + str(test_indel) + '_test_X_data_array.npy',X_data_array_test)
    np.save('rand_1_10fold/' + str(test_indel) + '_test_Y_data_array.npy',Y_data_array_test)
    np.save('rand_1_10fold/' + str(test_indel) + '_test_gene_pair_list_array.npy',gene_pair_list_array_test)
    np.save('rand_1_10fold/' + str(test_indel) + '_test_gene_pair_index_array.npy',gene_pair_index_array_test)

1
2
3
4
5
6
7
8
9
10


**TRAIN**

with PyG (PyTorch Geometric) library
# autocrine+ GCNG that uses both exocrine and autocrine gene interactions
#For diagonal GCNG, just feed it with a zero matrix instead of adjacent matrix.

In [ ]:
import os
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn import metrics
from scipy import interp

In [ ]:
# Load data
import numpy as np
from scipy import sparse as sp
import pickle
threshold = 140
with open('whole_FOV_distance_I_N_crs_140', 'rb') as fp:
    adj = pickle.load( fp)

In [ ]:
def degree_power(adj, pow):
    """
    Computes \(D^{p}\) from the given adjacency matrix. Useful for computing
    normalised Laplacians.
    :param adj: rank 2 array or sparse matrix
    :param pow: exponent to which elevate the degree matrix
    :return: the exponentiated degree matrix in sparse DIA format
    """
    degrees = np.power(np.array(adj.sum(1)), pow).flatten()
    degrees[np.isinf(degrees)] = 0.
    if sp.issparse(adj):
        D = sp.diags(degrees)
    else:
        D = np.diag(degrees)
    return D


def self_connection_normalized_adjacency(adj, symmetric=True):
    """
    Normalizes the given adjacency matrix using the degree matrix as either
    \(D~^{-1}A~\) or \(D~^{-1/2}A~D~^{-1/2}\) (symmetric normalization).where A~ = A+I
    :param adj: rank 2 array or sparse matrix;
    :param symmetric: boolean, compute symmetric normalization;
    :return: the normalized adjacency matrix.
    """
    if sp.issparse(adj):
        I = sp.eye(adj.shape[-1], dtype=adj.dtype)
    else:
        I = np.eye(adj.shape[-1], dtype=adj.dtype)
    A1 = adj + I
    if symmetric:
        normalized_D = degree_power(A1, -0.5)
        output = normalized_D.dot(A1).dot(normalized_D)
    else:
        normalized_D = degree_power(A1, -1.)
        output = normalized_D.dot(A1)
    return output

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)



In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [ ]:
!pip install torcheval

In [ ]:
from torcheval.metrics import BinaryAccuracy

In [ ]:
# Parameters
l2_reg = 5e-7  # Regularization rate for l2
learning_rate = 1*1e-6  # Learning rate for SGD
batch_size = 32  # Batch size
epochs = 5 # Number of training epochs
es_patience = 50  # Patience fot early stopping
criterion = torch.nn.BCELoss()

In [ ]:
# Model definition
class GCN(torch.nn.Module):
      def __init__(self):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(in_channels=2, out_channels=32, add_self_loops=False, bias=True)
        self.conv2 = GCNConv(in_channels= 32, out_channels=32, add_self_loops=False, bias=True)
        self.linear1 = torch.nn.Linear(29216,512)
        self.linear2 = torch.nn.Linear(512,1)

      def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = x.relu()
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, adj)
        x = x.relu()
        x = x.flatten()
        x = self.linear1(x)
        x = x.relu()
        x = self.linear2(x)
        x = x.sigmoid()
        return x

model = GCN()
print(model)

GCN(
  (conv1): GCNConv(2, 32)
  (conv2): GCNConv(32, 32)
  (linear1): Linear(in_features=29216, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=1, bias=True)
)


In [ ]:
for test_indel in range(1,3): ################## ten fold cross validation
    X_data_train = np.load('rand_1_10fold/'+str(test_indel)+'_train_X_data_array.npy')
    Y_data_train = np.load('rand_1_10fold/'+str(test_indel)+'_train_Y_data_array.npy')
    gene_pair_index_train = np.load('rand_1_10fold/'+str(test_indel)+'_train_gene_pair_list_array.npy')
    count_setx_train = np.load('rand_1_10fold/'+str(test_indel)+'_train_gene_pair_index_array.npy')
    X_data_test = np.load('rand_1_10fold/'+str(test_indel)+'_test_X_data_array.npy')
    Y_data_test = np.load('rand_1_10fold/'+str(test_indel)+'_test_Y_data_array.npy')
    gene_pair_index_test = np.load('rand_1_10fold/'+str(test_indel)+'_test_gene_pair_list_array.npy')
    count_set = np.load('rand_1_10fold/'+str(test_indel)+'_test_gene_pair_index_array.npy')
    trainX_index = [i for i in range(Y_data_train.shape[0])]
    validation_index = trainX_index[:int(np.ceil(0.2*len(trainX_index)))]
    train_index = trainX_index[int(np.ceil(0.2*len(trainX_index))):]
    X_train, y_train = X_data_train[train_index],Y_data_train[train_index][:,np.newaxis]
    X_val, y_val= X_data_train[validation_index],Y_data_train[validation_index][:,np.newaxis]
    X_test, y_test= X_data_test,Y_data_test[:,np.newaxis]

    print(type(y_train.flatten()))
    break


<class 'numpy.ndarray'>


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
for test_indel in range(1,3): ################## ten fold cross validation
    X_data_train = np.load('rand_1_10fold/'+str(test_indel)+'_train_X_data_array.npy')
    Y_data_train = np.load('rand_1_10fold/'+str(test_indel)+'_train_Y_data_array.npy')
    gene_pair_index_train = np.load('rand_1_10fold/'+str(test_indel)+'_train_gene_pair_list_array.npy')
    count_setx_train = np.load('rand_1_10fold/'+str(test_indel)+'_train_gene_pair_index_array.npy')
    X_data_test = np.load('rand_1_10fold/'+str(test_indel)+'_test_X_data_array.npy')
    Y_data_test = np.load('rand_1_10fold/'+str(test_indel)+'_test_Y_data_array.npy')
    gene_pair_index_test = np.load('rand_1_10fold/'+str(test_indel)+'_test_gene_pair_list_array.npy')
    count_set = np.load('rand_1_10fold/'+str(test_indel)+'_test_gene_pair_index_array.npy')
    trainX_index = [i for i in range(Y_data_train.shape[0])]
    validation_index = trainX_index[:int(np.ceil(0.2*len(trainX_index)))]
    train_index = trainX_index[int(np.ceil(0.2*len(trainX_index))):]
    X_train, y_train = X_data_train[train_index],Y_data_train[train_index][:,np.newaxis]
    X_val, y_val= X_data_train[validation_index],Y_data_train[validation_index][:,np.newaxis]
    X_test, y_test= X_data_test,Y_data_test[:,np.newaxis]

    # X_train, y_train, X_val, y_val, X_test, y_test, adj = mnist.load_data()
    # X_train, X_val, X_test = X_train[..., None], X_val[..., None], X_test[..., None]
    N = X_train.shape[-2]  # Number of nodes in the graphs
    F = X_train.shape[-1]  # Node features dimensionality
    n_out = y_train.shape[-1]  # Dimension of the target

    fltr = self_connection_normalized_adjacency(adj)

    fltr_tensor = sparse_mx_to_torch_sparse_tensor(fltr) # adj to tensor

    model = GCN()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)

    def train(x,y,A):
      model.train()
      optimizer.zero_grad()
      out = model(x, A)
      loss = criterion(out, y)
      loss.backward()
      optimizer.step()
      return out, loss.item()

    current_loss = 0
    all_losses = []


    for epoch in range(epochs):
      accuracy_train_list = []
      for i in range(len(X_train)):
        out, loss = train(torch.tensor(X_train[i], dtype=torch.float32), torch.tensor(y_train[i], dtype=torch.float32), fltr_tensor)
        current_loss += loss

        # training accuracy, threshold 0.5
        output_train_acc = out
        output_train_acc[output_train_acc >= 0.5] = 1
        output_train_acc[output_train_acc < 0.5] = 0
        output_train_acc_int = int(output_train_acc)   # float to int

        accuracy_train_list.append(output_train_acc_int)


      loss_epoch = current_loss / len(X_train)
      all_losses.append(loss_epoch)
      current_loss = 0

      accuracy_train = accuracy_score(y_train.flatten(), accuracy_train_list)
      print(f'Epoch: {epoch:03d}, Loss: {loss_epoch:.4f}, Accuracy: {accuracy_train:.4f}')
      #print(f'Epoch: {epoch:03d}, Loss: {loss_epoch:.4f}')


    overall_loss = sum(all_losses) / epochs
    print(f'OVERALL LOSS: {overall_loss:.4f}')
    break


    '''
    # Model definition
    X_in = Input(shape=(N, F))
    # Pass A as a fixed tensor, otherwise Keras will complain about inputs of
    # different rank.
    A_in = Input(tensor=sp_matrix_to_sp_tensor(fltr))

    graph_conv = GraphConv(32,activation='elu',kernel_regularizer=l2(l2_reg),use_bias=True)([X_in, A_in])
    graph_conv = GraphConv(32,activation='elu',kernel_regularizer=l2(l2_reg),use_bias=True)([graph_conv, A_in])
    flatten = Flatten()(graph_conv)
    fc = Dense(512, activation='relu')(flatten)
    output = Dense(n_out, activation='sigmoid')(fc)

    # Build model
    model = Model(inputs=[X_in, A_in], outputs=output)
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['acc'])
    model.summary()

    plot_model(model, to_file='gcn_LR_spatial_1.png', show_shapes=True)
    save_dir = current_path+'/'+str(test_indel)+'_self_connection_Ycv_LR_as_nega_rg_5-7_lr_1-6_e'+str(epochs)
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    early_stopping = EarlyStopping(monitor='val_acc', patience=es_patience, verbose=0, mode='auto')
    checkpoint1 = ModelCheckpoint(filepath=save_dir + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss',verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
    checkpoint2 = ModelCheckpoint(filepath=save_dir + '/weights.hdf5', monitor='val_acc', verbose=1,save_best_only=True, mode='auto', period=1)
    callbacks = [checkpoint2, early_stopping]

    # Train model
    validation_data = (X_val, y_val)
    history = model.fit(X_train,y_train,batch_size=batch_size,validation_data=validation_data,epochs=epochs,callbacks=callbacks)

    # Load best model
    # Save model and weights

    model_name = 'gcn_LR_1.h5'
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)
    # Score trained model.
    scores = model.evaluate(X_test, y_test, verbose=1,batch_size=batch_size)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    y_predict = model.predict(X_test)
    np.save(save_dir + '/end_y_test.npy', y_test)
    np.save(save_dir + '/end_y_predict.npy', y_predict)
    ############################################################################## plot training process

    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.grid()
    plt.legend(['train', 'val'], loc='upper left')
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.grid()
    plt.savefig(save_dir + '/end_result.pdf')
    ###############################################################
    #######################################

    #############################################################
    #########################
    y_testy = y_test
    y_predicty = y_predict
    fig = plt.figure(figsize=(5, 5))
    plt.plot([0, 1], [0, 1])
    plt.ylim([0, 1])
    plt.xlim([0, 1])
    plt.xlabel('FP')
    plt.ylabel('TP')
    # plt.grid()
    AUC_set = []
    s = open(save_dir + '/divided_interaction.txt', 'w')
    tprs = []
    mean_fpr = np.linspace(0, 1, 100)  # 3068
    for jj in range(len(count_set) - 1):  # len(count_set)-1):
        if count_set[jj] < count_set[jj + 1]:
            print(test_indel, jj, count_set[jj], count_set[jj + 1])
            y_test = y_testy[count_set[jj]:count_set[jj + 1]]
            y_predict = y_predicty[count_set[jj]:count_set[jj + 1]]
            # Score trained model.
            fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict, pos_label=1)
            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0
            # Print ROC curve
            plt.plot(fpr, tpr, color='0.5', lw=0.001, alpha=.2)
            auc = np.trapz(tpr, fpr)
            s.write(str(jj) + '\t' + str(count_set[jj]) + '\t' + str(count_set[jj + 1]) + '\t' + str(auc) + '\n')
            print('AUC:', auc)
            AUC_set.append(auc)

    mean_tpr = np.median(tprs, axis=0)
    mean_tpr[-1] = 1.0
    per_tpr = np.percentile(tprs, [25, 50, 75], axis=0)
    mean_auc = np.trapz(mean_tpr, mean_fpr)
    plt.plot(mean_fpr, mean_tpr, 'k', lw=3, label='median ROC')
    plt.title(str(mean_auc))
    plt.fill_between(mean_fpr, per_tpr[0, :], per_tpr[2, :], color='g', alpha=.2, label='Quartile')
    plt.plot(mean_fpr, per_tpr[0, :], 'g', lw=3, alpha=.2)
    plt.legend(loc='lower right')
    plt.savefig(save_dir + '/divided_interaction_percentile.pdf')
    del fig
    fig = plt.figure(figsize=(5, 5))
    plt.hist(AUC_set, bins=50)
    plt.savefig(save_dir + '/divided_interaction_hist.pdf')
    del fig
    s.close()'''

/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/_spmm.py:66: UserWarning: Converting sparse tensor to CSR format for more efficient processing. Consider converting your sparse tensor to CSR format beforehand to avoid repeated conversion (got 'torch.sparse_coo')
  warnings.warn(f"Converting sparse tensor to CSR format for more "


Epoch: 000, Loss: 0.6815, Accuracy: 0.5716


/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/_spmm.py:66: UserWarning: Converting sparse tensor to CSR format for more efficient processing. Consider converting your sparse tensor to CSR format beforehand to avoid repeated conversion (got 'torch.sparse_coo')
  warnings.warn(f"Converting sparse tensor to CSR format for more "


Epoch: 001, Loss: 0.6598, Accuracy: 0.6570


/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/_spmm.py:66: UserWarning: Converting sparse tensor to CSR format for more efficient processing. Consider converting your sparse tensor to CSR format beforehand to avoid repeated conversion (got 'torch.sparse_coo')
  warnings.warn(f"Converting sparse tensor to CSR format for more "


Epoch: 002, Loss: 0.6453, Accuracy: 0.6718


/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/_spmm.py:66: UserWarning: Converting sparse tensor to CSR format for more efficient processing. Consider converting your sparse tensor to CSR format beforehand to avoid repeated conversion (got 'torch.sparse_coo')
  warnings.warn(f"Converting sparse tensor to CSR format for more "


Epoch: 003, Loss: 0.6331, Accuracy: 0.6803


/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/_spmm.py:66: UserWarning: Converting sparse tensor to CSR format for more efficient processing. Consider converting your sparse tensor to CSR format beforehand to avoid repeated conversion (got 'torch.sparse_coo')
  warnings.warn(f"Converting sparse tensor to CSR format for more "


Epoch: 004, Loss: 0.6221, Accuracy: 0.6923
OVERALL LOSS: 0.6484


Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_train.flatten(), accuracy_train_list)
tn, fp, fn, tp = cm.ravel()

In [ ]:
print(cm)
print("True Positive :",tp)
print("True Negative :" ,tn)
print("False Positive :" ,fp)
print("False Negative :" ,fn)